# Recipe Recommender EDA Assignment

## Importing Libraries

In [1]:
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5,application_1676367681749_0006,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import SparkSession

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark = SparkSession.builder.appName("Case_Study").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# installing other required packages. 

spark.sparkContext.install_pypi_package("plotly==5.5.0")
spark.sparkContext.install_pypi_package("pandas==0.25.1")
spark.sparkContext.install_pypi_package("numpy==1.14.5")
spark.sparkContext.install_pypi_package("matplotlib==3.1.1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…



  Found existing installation: numpy 1.20.0
    Not uninstalling numpy at /usr/local/lib64/python3.7/site-packages, outside environment /mnt/yarn/usercache/livy/appcache/application_1676367681749_0006/container_1676367681749_0006_01_000001/tmp/1676377508776-0


The directory '/home/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
You are using pip version 9.0.1, however version 23.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

The directory '/home/.cache/pip/http' or its parent directory is not owned by the current user and the cache has 

In [6]:
from pyspark.sql import functions as F
from pyspark.ml.feature import Bucketizer

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from pyspark.sql.window import Window

# Import for typecasting columns
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType,StringType
from pyspark.sql.types import ArrayType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Reading the dataset

* **`TASK-01` --> Reading Recipes dataset**

In [7]:
# Reading raw_recipes dataset from s3 bucket
raw_recipes_df = spark.read.csv("s3://krishnademobucket-711/RAW_recipes_cleaned.csv", inferSchema=True, header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
raw_recipes_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+
|                name|    id|minutes|contributor_id| submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|
+--------------------+------+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+
|arriba   baked wi...|137739|     55|         47892|2005-09-16|['60-minutes-or-l...|[51.5, 0.0, 13.0,...|     11|['make a choice a...|autumn is my favo...|['winter squash',...|            7|
|a bit different  ...| 31490|     30|         26278|2002-06-17|['30-minutes-or-l...|[173.4, 18.0, 0.0...|      9|['preheat oven to...|this recipe calls...|['prepared pizza ...|            6|
|all in the kitche...|112140|    130|        

In [9]:
# Size of the dataset
raw_recipes_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

231637

In [10]:
# number of columns in dataset
len(raw_recipes_df.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12

In [11]:
raw_recipes_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- contributor_id: integer (nullable = true)
 |-- submitted: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- nutrition: string (nullable = true)
 |-- n_steps: integer (nullable = true)
 |-- steps: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: string (nullable = true)
 |-- n_ingredients: integer (nullable = true)

In [12]:
# checking null values in the dataset
raw_recipes_df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in raw_recipes_df.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+---+-------+--------------+---------+----+---------+-------+-----+-----------+-----------+-------------+
|name| id|minutes|contributor_id|submitted|tags|nutrition|n_steps|steps|description|ingredients|n_ingredients|
+----+---+-------+--------------+---------+----+---------+-------+-----+-----------+-----------+-------------+
|   1|  0|      0|             0|        0|   0|        0|      0|    0|          0|          0|            0|
+----+---+-------+--------------+---------+----+---------+-------+-----+-----------+-----------+-------------+

In [13]:
# List of nutrition columns

nutrition_column_names = ['calories', 'total_fat_PDV', 'sugar_PDV', 'sodium_PDV', 'protein_PDV', 'saturated_fat_PDV',
                          'carbohydrates_PDV']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

* **`TASK-02` --> Removing single quotes and square brackets from nutrition values and assigning these values to different nutrition columns**

In [14]:
# Extracting individual values from nutrition column

raw_recipes_df = (raw_recipes_df.withColumn('nutrition',(F.regexp_replace("nutrition","[\[\]]", ""))))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# now we need to split the values in nutrition columns and assign it to different nutritiocolumn_names
nutrition_cols_split = F.split(raw_recipes_df['nutrition'], ',' )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# assigning the values to nutrition columns and changing data type to float
for col_index, col_name in enumerate(nutrition_column_names):
    raw_recipes_df = (raw_recipes_df.withColumn(col_name, nutrition_cols_split.getItem(col_index).cast("float")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
raw_recipes_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------+-------------+---------+----------+-----------+-----------------+-----------------+
|                name|    id|minutes|contributor_id| submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|calories|total_fat_PDV|sugar_PDV|sodium_PDV|protein_PDV|saturated_fat_PDV|carbohydrates_PDV|
+--------------------+------+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------+-------------+---------+----------+-----------+-----------------+-----------------+
|arriba   baked wi...|137739|     55|         47892|2005-09-16|['60-minutes-or-l...|51.5, 0.0, 13.0, ...|     11|['make a choice a...|autumn is my favo

In [18]:
# using assert commands checking the code 

assert raw_recipes_df.schema["protein_PDV"].dataType == FloatType(), "Recheck your typecasting"
assert raw_recipes_df.collect()[123432][14] == 62.0, "The columns have not been split correctly."
assert raw_recipes_df.collect()[10000][12] == 60.400001525878906, "The columns have not been split correctly."

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

* **`TASK-03` -> Standardising the nutrition values**

In [19]:
# Standardising the nutrition values using calories as base of standardisation and filling null values

col_name_list = []
for nutrition_col in nutrition_column_names:# loop over each of the newly created nutrition columns    
    if nutrition_col != 'calories': # the calories column should not be a part of the transformation exercise
        # following code will name the new columns 
        nutrition_per_100_cal_col = (nutrition_col.replace('_PDV','')+'_per_100_cal')
        col_name_list.append(nutrition_per_100_cal_col)
        raw_recipes_df = raw_recipes_df.withColumn(nutrition_per_100_cal_col,F.col(nutrition_col)*100/ F.col('calories'))
        raw_recipes_df = raw_recipes_df.na.fill(value=0,subset= col_name_list)  

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
raw_recipes_df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------+-------------+---------+----------+-----------+-----------------+-----------------+---------------------+------------------+------------------+-------------------+-------------------------+-------------------------+
|                name|    id|minutes|contributor_id| submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|calories|total_fat_PDV|sugar_PDV|sodium_PDV|protein_PDV|saturated_fat_PDV|carbohydrates_PDV|total_fat_per_100_cal| sugar_per_100_cal|sodium_per_100_cal|protein_per_100_cal|saturated_fat_per_100_cal|carbohydrates_per_100_cal|
+--------------------+------+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+---------------

In [22]:
# Using same assert commands provided in the sample template

# total fat check for id 28881
assert raw_recipes_df.filter("id == 28881").select('total_fat_per_100_cal').first()[0] == 0, "total_fat_per_100_cal for recipe 28881 should be 0"

# total fat check for id 112140
assert round(raw_recipes_df.filter("id == 112140").select('total_fat_per_100_cal').first()[0]) == 8, "total_fat_per_100_cal for recipe 112140 should be 8"

# checking for nulls
for c in ['total_fat_per_100_cal','sugar_per_100_cal','sodium_per_100_cal','protein_per_100_cal',
                          'saturated_fat_per_100_cal','carbohydrates_per_100_cal']:
    assert raw_recipes_df.select(F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c)).collect()[0][0] == 0, "There are Nulls in the data"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

* **`TASK-04` --> Converting the tags column from string to array of strings**

In [23]:
# typecasting the tags feature from string to Array of strings and removing single quotes & square brackets
# If necessary we can use same type casting for ingredients feature
raw_recipes_df = (raw_recipes_df.withColumn('tags', (F.regexp_replace("tags","[\[\]'']", "")))
                  .withColumn('tags', F.split(F.col("tags"), ", ").cast(ArrayType(StringType())).alias("tags")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# typecasting the ingredients feature from string to Array of strings and removing single quotes & square brackets
raw_recipes_df = (raw_recipes_df.withColumn('ingredients', (F.regexp_replace("ingredients","[\[\]'']", "")))
                  .withColumn('ingredients', F.split(F.col("ingredients"), ", ").cast(ArrayType(StringType())).alias("ingredients")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
raw_recipes_df.collect()[1]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(name='a bit different  breakfast pizza', id=31490, minutes=30, contributor_id=26278, submitted='2002-06-17', tags=['30-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'breakfast', 'main-dish', 'pork', 'american', 'oven', 'easy', 'kid-friendly', 'pizza', 'dietary', 'northeastern-united-states', 'meat', 'equipment'], nutrition='173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0', n_steps=9, steps="['preheat oven to 425 degrees f', 'press dough into the bottom and sides of a 12 inch pizza pan', 'bake for 5 minutes until set but not browned', 'cut sausage into small pieces', 'whisk eggs and milk in a bowl until frothy', 'spoon sausage over baked crust and sprinkle with cheese', 'pour egg mixture slowly over sausage and cheese', 's& p to taste', 'bake 15-20 minutes or until eggs are set and crust is brown']", description='this recipe calls for the crust to be prebaked a bit before adding ingredients. feel free to change sausage

In [26]:
# using assert commands checking the code (assert commands are taken from sample template)  

assert raw_recipes_df.schema["tags"].dataType == ArrayType(StringType(), True), "You have not split the string into an array."
assert raw_recipes_df.collect()[2][5] == ['time-to-make','course', 'preparation', 'main-dish', 'chili', 'crock-pot-slow-cooker', 'dietary', 'equipment', '4-hours-or-less'], "Recheck your string cleaning and splitting operations."

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# number of columns in finalised dataframe
len(raw_recipes_df.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

25

In [28]:
raw_recipes_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- contributor_id: integer (nullable = true)
 |-- submitted: string (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- nutrition: string (nullable = true)
 |-- n_steps: integer (nullable = true)
 |-- steps: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- n_ingredients: integer (nullable = true)
 |-- calories: float (nullable = true)
 |-- total_fat_PDV: float (nullable = true)
 |-- sugar_PDV: float (nullable = true)
 |-- sodium_PDV: float (nullable = true)
 |-- protein_PDV: float (nullable = true)
 |-- saturated_fat_PDV: float (nullable = true)
 |-- carbohydrates_PDV: float (nullable = true)
 |-- total_fat_per_100_cal: double (nullable = false)
 |-- sugar_per_100_cal: double (nullable = false)
 |-- sodiu

## Reading second dataset as RAW_ratings_df

* **`TASK-05` --> Reading Rating dataset and joining it with recipes dataset**

In [29]:
# reading the second dataset
raw_ratings_df = spark.read.csv("s3://krishnademobucket-711/RAW_interactions_cleaned.csv", inferSchema=True, header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
# Checking Ratings dataframe
raw_ratings_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------+----------+------+--------------------+
|   user_id|recipe_id|      date|rating|              review|
+----------+---------+----------+------+--------------------+
|     38094|    40893|2003-02-17|     4|Great with a sala...|
|   1293707|    40893|2011-12-21|     5|So simple  so del...|
|      8937|    44394|2002-12-01|     4|This worked very ...|
|    126440|    85009|2010-02-27|     5|I made the Mexica...|
|     57222|    85009|2011-10-01|     5|Made the cheddar ...|
|     52282|   120345|2005-05-21|     4|very very sweet. ...|
|    124416|   120345|2011-08-06|     0|Just an observati...|
|2000192946|   120345|2015-05-10|     2|This recipe was O...|
|     76535|   134728|2005-09-02|     4|          Very good!|
|    273745|   134728|2005-12-22|     5|Better than the r...|
|    353911|   134728|2006-09-26|     5|Absolutely AWESOM...|
|    190375|   134728|2007-03-09|     5|These taste absol...|
|    468945|   134728|2008-02-20|     0|Made my own butte...|
|    255

In [31]:
# creating newcolumn with "review_date" name which is same as "date" and drop "date" column
raw_ratings_df = raw_ratings_df.withColumn('review_date',F.col('date')).drop(F.col('date'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
raw_ratings_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------+------+--------------------+-----------+
|   user_id|recipe_id|rating|              review|review_date|
+----------+---------+------+--------------------+-----------+
|     38094|    40893|     4|Great with a sala...| 2003-02-17|
|   1293707|    40893|     5|So simple  so del...| 2011-12-21|
|      8937|    44394|     4|This worked very ...| 2002-12-01|
|    126440|    85009|     5|I made the Mexica...| 2010-02-27|
|     57222|    85009|     5|Made the cheddar ...| 2011-10-01|
|     52282|   120345|     4|very very sweet. ...| 2005-05-21|
|    124416|   120345|     0|Just an observati...| 2011-08-06|
|2000192946|   120345|     2|This recipe was O...| 2015-05-10|
|     76535|   134728|     4|          Very good!| 2005-09-02|
|    273745|   134728|     5|Better than the r...| 2005-12-22|
|    353911|   134728|     5|Absolutely AWESOM...| 2006-09-26|
|    190375|   134728|     5|These taste absol...| 2007-03-09|
|    468945|   134728|     0|Made my own butte...| 2008

In [33]:
raw_ratings_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: integer (nullable = true)
 |-- recipe_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review: string (nullable = true)
 |-- review_date: string (nullable = true)

In [34]:
# Shape of the Ratings dataset
print("size of the Ratings dataset:",raw_ratings_df.count())
print("Number of columns in Ratings dataset:",len(raw_ratings_df.columns))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

size of the Ratings dataset: 1132367
Number of columns in Ratings dataset: 5

In [35]:
## Joining the RAW_ratings_df and RAW_recipes_df
interaction_level_df = raw_ratings_df.join(raw_recipes_df, raw_ratings_df.recipe_id==raw_recipes_df.id,'outer')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
# Shape of the overall dataframe
print("Size of final dataset:",interaction_level_df.count())
print("Number of columns in final dataset:",len(interaction_level_df.columns))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Size of final dataset: 1132367
Number of columns in final dataset: 30

In [37]:
interaction_level_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+------+--------------------+-----------+--------------------+---+-------+--------------+----------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+-------------+--------+-------------+---------+----------+-----------+-----------------+-----------------+---------------------+-------------------+------------------+-------------------+-------------------------+-------------------------+
|user_id|recipe_id|rating|              review|review_date|                name| id|minutes|contributor_id| submitted|                tags|           nutrition|n_steps|               steps|         description|         ingredients|n_ingredients|calories|total_fat_PDV|sugar_PDV|sodium_PDV|protein_PDV|saturated_fat_PDV|carbohydrates_PDV|total_fat_per_100_cal|  sugar_per_100_cal|sodium_per_100_cal|protein_per_100_cal|saturated_fat_per_100_cal|carbohydrates_per_100_cal|
+-------+---------+------+--------------------+-----------

In [38]:
interaction_level_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: integer (nullable = true)
 |-- recipe_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review: string (nullable = true)
 |-- review_date: string (nullable = true)
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- contributor_id: integer (nullable = true)
 |-- submitted: string (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- nutrition: string (nullable = true)
 |-- n_steps: integer (nullable = true)
 |-- steps: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- n_ingredients: integer (nullable = true)
 |-- calories: float (nullable = true)
 |-- total_fat_PDV: float (nullable = true)
 |-- sugar_PDV: float (nullable = true)
 |-- sodium_PDV: float (nullable = true)
 |-- protein_PDV: float (nullable = true)
 |-- sat

In [39]:
# using same assert commands given in template

assert (interaction_level_df.count() ,len(interaction_level_df.columns)) == (1132367, 30), "The type of join is incorrect"

list1 = raw_ratings_df.select('recipe_id').collect()
list2 = raw_recipes_df.select('id').collect()
exclusive_set = set(list1)-set(list2)

assert len(exclusive_set) == 0, "There is a mistake in reading one of the two data files."

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

* **From the schema we know that submitted and review_date are of String datatypes but we need to convert or type cast it into date datatype.**

* **`TASK-06` --> Converting columns to correct datatype and creating time based features**

In [40]:
# type casting submitted and review_date features into date type and creating other time based
interaction_level_df = (interaction_level_df.withColumn('submitted',F.col("submitted").cast("date"))
                        .withColumn('review_date',F.col("review_date").cast("date")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
interaction_level_df = (interaction_level_df.withColumn('days_since_submission_on_review_date',F.datediff("review_date","submitted"))
                        .withColumn('months_since_submission_on_review_date',F.months_between("review_date","submitted"))
                        .withColumn('years_since_submission_on_review_date',F.months_between("review_date","submitted")/12))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
print(interaction_level_df.count(),  len(interaction_level_df.columns))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1132367 33

In [43]:
# Using same assert commands from Sample template

assert interaction_level_df.schema["days_since_submission_on_review_date"].dataType == IntegerType()

assert (interaction_level_df.filter((interaction_level_df.user_id == 428885) & (interaction_level_df.recipe_id == 335241))
                            .select('days_since_submission_on_review_date').collect()[0][0]) == 77
assert (interaction_level_df.filter((interaction_level_df.user_id == 2025676) & (interaction_level_df.recipe_id == 94265))
                            .select('months_since_submission_on_review_date').collect()[0][0]) == 153.22580645
assert (interaction_level_df.filter((interaction_level_df.user_id == 338588) & (interaction_level_df.recipe_id == 21859))
                            .select('years_since_submission_on_review_date').collect()[0][0]) == 4.564516129166667

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
# Schema of Overall dataframe
interaction_level_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- user_id: integer (nullable = true)
 |-- recipe_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review: string (nullable = true)
 |-- review_date: date (nullable = true)
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- minutes: integer (nullable = true)
 |-- contributor_id: integer (nullable = true)
 |-- submitted: date (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- nutrition: string (nullable = true)
 |-- n_steps: integer (nullable = true)
 |-- steps: string (nullable = true)
 |-- description: string (nullable = true)
 |-- ingredients: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- n_ingredients: integer (nullable = true)
 |-- calories: float (nullable = true)
 |-- total_fat_PDV: float (nullable = true)
 |-- sugar_PDV: float (nullable = true)
 |-- sodium_PDV: float (nullable = true)
 |-- protein_PDV: float (nullable = true)
 |-- saturat

In [47]:
# Checking for Null values present in overall_df
interaction_level_df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in interaction_level_df.columns]).show()

# There are no null values apart from name feature which has 1 null value in it.

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------+------+------+-----------+----+---+-------+--------------+---------+----+---------+-------+-----+-----------+-----------+-------------+--------+-------------+---------+----------+-----------+-----------------+-----------------+---------------------+-----------------+------------------+-------------------+-------------------------+-------------------------+------------------------------------+--------------------------------------+-------------------------------------+
|user_id|recipe_id|rating|review|review_date|name| id|minutes|contributor_id|submitted|tags|nutrition|n_steps|steps|description|ingredients|n_ingredients|calories|total_fat_PDV|sugar_PDV|sodium_PDV|protein_PDV|saturated_fat_PDV|carbohydrates_PDV|total_fat_per_100_cal|sugar_per_100_cal|sodium_per_100_cal|protein_per_100_cal|saturated_fat_per_100_cal|carbohydrates_per_100_cal|days_since_submission_on_review_date|months_since_submission_on_review_date|years_since_submission_on_review_date|
+-------+-------

In [48]:
## checking statistical values for newly created time based feaures
interaction_level_df.select(['days_since_submission_on_review_date','months_since_submission_on_review_date',
                   'years_since_submission_on_review_date']).describe().show()

# least value is -1765 days (days cannot be in negative values) & max value is 7044 days.

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------------------------+--------------------------------------+-------------------------------------+
|summary|days_since_submission_on_review_date|months_since_submission_on_review_date|years_since_submission_on_review_date|
+-------+------------------------------------+--------------------------------------+-------------------------------------+
|  count|                             1132367|                               1132367|                              1132367|
|   mean|                  1389.9705952222203|                     45.66450675099399|                   3.8053755625830186|
| stddev|                  1282.3787264985817|                     42.13040604045353|                   3.5108671700377974|
|    min|                               -1765|                                 -58.0|                   -4.833333333333333|
|    max|                                7044|                          231.41935484|                   19.284946236666666|
+-------

In [49]:
interaction_level_df.write.mode('overwrite').parquet('s3://krishnademobucket-711/interaction_level_df_final_Processed_data.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…